In [1]:
import cv2 as cv                
from skimage import feature  
import os
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
def compute_hog_train(img):
    img = cv.resize(img, (128, 256))
    (hog, hog_image) = feature.hog(img, orientations=9,
                                   pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                                   block_norm='L2-Hys', visualize=True, transform_sqrt=True)
    return hog

In [3]:
def load_data(non_pedestrian_folder, pedestrian_folder):
    features = []
    labels = []
    
    # Load ảnh NonPedestrian
    for file_name in os.listdir(non_pedestrian_folder):
        if file_name.endswith('.jpg'):
            img_path = os.path.join(non_pedestrian_folder, file_name)
            img = cv.imread(img_path)
            if img is not None:
                img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                feature = compute_hog_train(img_gray)
                features.append(feature)
                labels.append(0)  # NonPedestrian nhãn là 0
    
    # Load ảnh Pedestrian
    for file_name in os.listdir(pedestrian_folder):
        if file_name.endswith('.jpg'):
            img_path = os.path.join(pedestrian_folder, file_name)
            img = cv.imread(img_path)
            if img is not None:
                img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                feature = compute_hog_train(img_gray)
                features.append(feature)
                labels.append(1)  # Pedestrian nhãn là 1

    return np.array(features), np.array(labels)

In [4]:
features, labels = load_data('NonPedestrian', 'Pedestrian')
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=42)

In [5]:
for k in range(1, 11):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy for k={k}: {accuracy}')

Accuracy for k=1: 0.75
Accuracy for k=2: 0.6875
Accuracy for k=3: 0.75
Accuracy for k=4: 0.625
Accuracy for k=5: 0.75
Accuracy for k=6: 0.6875
Accuracy for k=7: 0.75
Accuracy for k=8: 0.625
Accuracy for k=9: 0.8125
Accuracy for k=10: 0.75


In [6]:
logistic_model = LogisticRegression(max_iter=50)
logistic_model.fit(X_train, y_train)
y_pred = logistic_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of Logistic Regression: {accuracy}")

Accuracy of Logistic Regression: 0.9375


Vậy LogisticRegression trong trường hợp này sẽ cho kết quả chính xác hơn KNN(k từ 1 tới 10)